# ICM25

Analysis of the long-term evolution of international equity markets (average returns, volatilities, long-term correlations) and economic risks of European stock markets.

## Case 1: Analyzing 25 years of stock market returns

Analysis of the **longterm evolution of international equity markets**, i.e.
- Average returns
- Volatilities
- Long-term correlations between markets
- Correlation regimes

Data set ([Case1.csv](./data/Case1.csv)) includes 16 stock market indices in local currencies covering developed markets in North America, Europe and Asia-Pacific
- Monthly stock market data starting on December 31, 1992, and ending on February 28, 2018
- Indexed to ”100” at the beginning of the period

In [2]:
import math
import numpy as np
import pandas as pd
import plotly.express as px

In [3]:
# Metadata
data = pd.read_csv('data/Case1.csv')
# print(data.info())
# print(data.describe())
data['Date'] = pd.to_datetime(data['Date'])

1. Display the long-term performance of the equity markets

In [4]:
data_melted = data.melt(id_vars=['Date'], value_vars=data.columns[1:], var_name='Index', value_name='Value')
fig = px.line(data_melted, x='Date', y='Value', color='Index', title='Long-term performance of the equity markets', log_y=True)
fig.show()

2. Calculate mean returns and volatilities over the total period

In [5]:
returns = data.copy()
for index in data.columns[1:]:
    for i in range(1, len(data[index])):
        returns.loc[i, index] = np.log(data.loc[i, index] / data.loc[i - 1, index])

# First row needs to be removed as it is 100% return
returns = returns.drop(0)

# print(returns.info())
# print(returns.head())

# Mean and SD (volatility) of the returns
results_list = []
for index in returns.columns[1:]:
    mean_return = math.exp(returns[index].mean()*12)-1
    std_return = returns[index].std()*math.sqrt(12)
    results_list.append({'Index': index, 'Mean Return p.a.': mean_return, 'Volatility': std_return})

results = pd.DataFrame(results_list)
results_melted = results.melt(id_vars='Index', value_vars=['Mean Return p.a.', 'Volatility'], var_name='Metric', value_name='Value')
fig = px.bar(results_melted, x='Index', y='Value', color='Metric', barmode='group', title='Mean returns p.a. and volatilities')
fig.show()

3. Calculate yearly returns for the stock markets


In [6]:
returns['Year'] = returns['Date'].dt.year
grouped_returns = returns.drop(columns=['Date']).groupby('Year').sum().reset_index()

fig = px.line(grouped_returns, x='Year', y=grouped_returns.columns[1:], title='Yearly returns for the stock markets')
fig.show()

4. Calculate the correlations between the markets over the period 1993-2017


In [7]:
correlation_matrix = returns.drop(columns=['Date', 'Year']).corr()
correlation_matrix.loc['Mean'] = correlation_matrix.mean(axis=1)

fig = px.imshow(correlation_matrix, text_auto=True, title='Correlation Matrix (full)', color_continuous_scale='RdBu_r')
fig.show()

5. Calculate the correlations for five sub-periods and analyze the correlation regimes
- 1993-1997
- 1998-2002
- 2003-2007
- 2008-2012
- 2013-2017

In [8]:
correlation_matrix_avg = pd.DataFrame()

time_delta = 4
for year in range(1993, 2018, time_delta):
    returns_sub = returns[returns['Year'].between(year, year+time_delta)]
    correlation_matrix = returns_sub.drop(columns=['Date', 'Year']).corr()
    correlation_matrix_avg[f'{year}-{year + time_delta}'] = correlation_matrix.mean()

    fig = px.imshow(correlation_matrix, text_auto=True, title=f'Correlation Matrix ({year}-{year + time_delta})', color_continuous_scale='RdBu_r', zmin=0, zmax=1)
    fig.show()

fig = px.imshow(correlation_matrix_avg, text_auto=True, title='Average correlation over sub-periods', color_continuous_scale='RdBu_r', zmin=0, zmax=1)
fig.show()

## Case 2: Rational asset pricing and multifactor models

Data set ([Case2Factors.csv](./data/Case2Factors.csv) and [Case2MSCI.csv](./data/Case2MSCI.csv)) include monthly total return index data over the 1st decade of the 21st century, from January 2000 to December 2009, denominated in EUR for 10 European stock markets and 4 global risk factors.

[Case2.py](./case2.py) analyzes the **relationships between the returns of the stock markets and the changes of the global factors** using the following regressions for each market:
- **Market return on the MSCI World index return** (_single factor model_)
- **Market return on the 4 global factors** (_4-factor model_)

In [9]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

Prepare MSCI world return data (Switzerland, Germany, France, UK, Japan, USA)

In [10]:
dataMSCI = pd.read_csv('data/Case2MSCI.csv')
# print(dataMSCI.info())
# print(dataMSCI.describe())
dataMSCI['Date'] = pd.to_datetime(dataMSCI['Date'])

returnsMSCI = dataMSCI.copy()
for index in dataMSCI.columns[1:]:
    for i in range(1, len(dataMSCI[index])):
        returnsMSCI.loc[i, index] = np.log(dataMSCI.loc[i, index] / dataMSCI.loc[i - 1, index])

# First row needs to be removed as it is 100% return
returnsMSCI = returnsMSCI.drop(0)

# print(returnsMSCI.info())
# print(returnsMSCI.head())

Prepare global factors return data (MSCI World, CRB Index, EUR 10Y Rate, FX USD/EUR)

In [11]:
dataFactors = pd.read_csv('data/Case2Factors.csv')
# print(dataFactors.info())
# print(dataFactors.describe())
dataFactors['Date'] = pd.to_datetime(dataFactors['Date'])

returnsFactors = dataFactors.copy()
for index in dataFactors.columns[1:]:
    for i in range(1, len(dataFactors[index])):
        returnsFactors.loc[i, index] = np.log(dataFactors.loc[i, index] / dataFactors.loc[i - 1, index])

# First row needs to be removed as it is 100% return
returnsFactors = returnsFactors.drop(0)

# print(returnsFactors.info())
# print(returnsFactors.head())

Market return on the MSCI World index return (single factor model)

In [12]:
for country in returnsMSCI.columns[1:]:
    y = returnsMSCI[country]
    X = returnsFactors['MSCI World']
    # Add a constant to the independent variable
    X = sm.add_constant(X)

    model = sm.OLS(y, X).fit()
    print(f"\n\n{"="*32} {country} {"="*33}")
    print(model.summary())



================================ Switzerland =================================
                            OLS Regression Results                            
Dep. Variable:            Switzerland   R-squared:                       0.631
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     202.0
Date:                Sun, 11 Aug 2024   Prob (F-statistic):           2.50e-27
Time:                        22:48:25   Log-Likelihood:                 273.35
No. Observations:                 120   AIC:                            -542.7
Df Residuals:                     118   BIC:                            -537.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------

Market return on the 4 global factors (4-factor model)

In [13]:
for country in returnsMSCI.columns[1:]:
    y = returnsMSCI[country]
    X = returnsFactors[['FX USD/EUR', 'EUR 10Y Rate' ,'CRB Index', 'MSCI World']]
    # Add a constant to the independent variable
    X = sm.add_constant(X)

    model = sm.OLS(y, X).fit()
    print(f"\n\n{"="*32} {country} {"="*33}")
    print(model.summary())



================================ Switzerland =================================
                            OLS Regression Results                            
Dep. Variable:            Switzerland   R-squared:                       0.674
Model:                            OLS   Adj. R-squared:                  0.662
Method:                 Least Squares   F-statistic:                     59.37
Date:                Sun, 11 Aug 2024   Prob (F-statistic):           4.25e-27
Time:                        22:48:25   Log-Likelihood:                 280.69
No. Observations:                 120   AIC:                            -551.4
Df Residuals:                     115   BIC:                            -537.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------